In [ ]:
import json
# from together import Together
from openai import OpenAI

# togetherai_key = <insert togetherai key>
together_client = Together(api_key=togetherai_key)

# openai_key = <insert openai key>
client = OpenAI(api_key=openai_key)

In [2]:
# Loading all JSONs

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/ref_extractions.json', 'r') as f:
    ref_extractions = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/case_summary_pairs_list.json', 'r') as f:
    case_summary_pairs_list = json.load(f)

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/matching_uids_xml_paths.json', 'r') as f:
    matching_uids_xml_paths = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/citation_extrac_list.json', 'r') as f:
    citation_list = json.load(f)

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/matching_uids_pmid_list.json', 'r') as f:
    matching_uids_pmid_list = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/o3_mini_response_list.json', 'r') as f:
    o3_mini_response_list = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/indices_list.json', 'r') as f:
    citation_valid_indices = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/generated_questions.json', 'r') as f:
    generated_questions = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/summaries_valid_indices.json', 'r') as g:
    summaries_valid_indices = json.load(g)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/common_indices.json', 'r') as g:
    common_indices = json.load(g)

In [3]:
### citation_valid_indices_dict was not saved as json because it turns the keys into strings (instead of ints) ###
citation_valid_indices_dict = {citation_valid_indices[idx]: idx for idx in range(len(citation_valid_indices))}

* o3_mini_response_list:
    * Length - 31,080
    * Content - The reason for citation between each similar patient pair
    * Recall that len(matching_uids) is 31,692, hence 612 pairs could not be extracted in o3_mini_response_list.

* citation_valid_indices:
    * Length - 31,080
    * Content - Created for index alignment between matching_uids and o3_mini_response_list.
      E.g., citation_valid_indices[0] == 0, citation_valid_indices[31079] == 31691

* citation_valid_indices_dict:
    * Length - 31,080
    * Content -
        * Keys - The index in matching_uids
        * Values - The corresponding index in o3_mini_response_list

* case_summary_pairs_list:
    * Length - 31,692
    * Content - A list in which each pair represents the two summaries of similar patients A and B.
    * 83 elements contain 0/1 summaries, we want to drop them out.
    * Hence, the actual number of valid summaries is 31,609.

* common_indices:
    * Length - 31,002
    * Content - Contains all of the valid indices (0-31,691) recognized by both citation and summary extraction.

* generated_questions:
    * Length - 31,002
    * Content - For each valid common index, contains the generated questions given case1, case2 and citation reason.
    
Why did we write all that? For answer generation prompt, we'll have:

1. curr_index = common_indices[11135]

2. case1 = case_summary_pairs_list[curr_index][0]

3. case2 = case_summary_pairs_list[curr_index][1]

4. citation_reason = o3_mini_response_list[citation_valid_indices_dict[curr_index]]

5. curr_questions = generated_questions[11135]

Why in 'citation_reason', we do:  
'o3_mini_response_list[citation_valid_indices_dict[curr_index]]'  
Instead of just:  
'o3_mini_response_list[curr_index]'?  

Because curr_index is the aligned index (between 0 and 31,691), while o3_mini_response_list contains less than 31,692 elements (31,080).
We need that dictionary to map between the aligned index, and the actual index in o3_mini_response_list.


Why in 'curr_questions' we use the same index as used in 'common_indices'?

Because 'curr_questions' and 'common_indices' are of the same length. As long we go over each INDEX of 'common_indices' (0-31,001),
rather than each ELEMENT of 'common_indices' (0-31,691) - we keep the alignment.

In [4]:
len(common_indices)

31002

In [5]:
idx = 101
common_index = common_indices[idx]

case1 = case_summary_pairs_list[common_index][0]

case2 = case_summary_pairs_list[common_index][1]

citation_reason = o3_mini_response_list[citation_valid_indices_dict[common_index]]

curr_questions = generated_questions[idx]

answering_prompt = """You will be given case details for Case A, Case B, Reason for Citation (one case cites the other), along with a list of questions.
You are an experienced clinician specializing in medical imaging. Previously, a list of comparative, clinically oriented questions was generated for the two cases.
Now, your task is to answer those questions in a way that highlights the diagnostic similarities and differences between the two cases.
DO NOT provide any additional information or context outside of the answers to the questions.
"""
# Instructions for answering:

# 1. **Refer to Both Cases**  
#    - You will be given the same case details for Case A, Case B, Reason for Citation, along with the full list of questions.
#    - Answer each question by explicitly referencing the relevant details from Case A and Case B that support your conclusion.

# 2. **Structure of Answers**  
#    - For yes/no questions, begin your response with a clear “Yes” or “No,” then provide a brief rationale.
#    - For open-ended or more detailed comparative questions, give a concise yet clinically thorough answer.
#    - If you need to highlight differences in imaging findings, symptoms, or etiologies, do so clearly and directly.
#    - Split each answer to 'In case A, ...' and 'In case B, ...'

# 3. **Clinical Depth & Relevance**  
#    - Focus on the clinical significance of each answer, referencing imaging features, pathophysiology, or typical clinical presentation.
#    - Avoid tangential or purely theoretical discussion; make sure each answer directly relates to the question and helps differentiate or confirm diagnoses.

# 4. **Accuracy & Clarity**  
#    - Base your answers on standard radiological and clinical knowledge, ensuring that any assertions are consistent with typical findings and established practice.
#    - If you must make an assumption (e.g., missing data), state it explicitly and provide your best inference based on standard clinical reasoning.

# 5. **Format**  
#    - Number your answers to match the questions.
#    - Keep them concise, but complete enough to stand as a definitive answer.
# """

answering_user = f"""Below are the two case descriptions and the set of questions to be answered.

---
Case A:
{case1}

Case B:
{case2}

Reason for Citation, Possible Similarities and Explanation:
{citation_reason}

Questions:
{curr_questions}
---

Provide your answers now:
"""


In [6]:
print(answering_user)

Below are the two case descriptions and the set of questions to be answered.

---
Case A:
A 48 year old male presented to our emergency department with pain and restriction of movements involving both shoulders following a fall. A thorough history into the mechanism of fall revealed that the patient was a known diabetic on oral hypoglycaemic drugs and following a syncopal attack, he fell on his outstretched right hand with the body weight falling on it, causing it to dislocate posteriorly. And a bystander in trying to prevent the patient from falling held on to his left upper limb, which was pulled into abduction, extension and external rotation producing an anterior dislocation on the left side. A detailed clinical examination showed flattening of deltoid contours on both sides. Humeral head was palpable posteriorly on the right side and anteriorly on the left side. Surprisingly, there were no characteristic deformities with the arms hanging by the side of the body. Attempted range of

In [8]:
lst1 = []
for i in range(len(common_indices))[:len(common_indices)//2]:
    lst1.append(i)


lst2 = []
for i in range(len(common_indices))[len(common_indices)//2:]:
    lst2.append(i)


lst1[-1]

15500

In [9]:
with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/batch_requests_answer_gen_part1.jsonl", "w", encoding="utf-8") as f:
    for idx in range(len(common_indices))[:len(common_indices)//2]:
        common_index = common_indices[idx]
        case1 = case_summary_pairs_list[common_index][0]
        case2 = case_summary_pairs_list[common_index][1]
        citation_reason = o3_mini_response_list[citation_valid_indices_dict[common_index]]
        curr_questions = generated_questions[idx]

        answering_user = f"""Below are the two case descriptions and the set of questions to be answered.

        ---
        Case A:
        {case1}

        Case B:
        {case2}

        Reason for Citation, Possible Similarities and Explanation:
        {citation_reason}

        Questions:
        {curr_questions}
        ---

        Provide your answers now:
        """

        request_obj = {
            "custom_id": f"request-{idx+1}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                    "model": "gpt-4.1-mini",
                    "messages": [
                        {
                            "role": "developer",
                            "content": f"{answering_prompt}"
                        },
                        {
                            "role": "user",
                            "content": f"{answering_user}"
                        }
                    ],
            }
        }
        # Write each object on a new line in the JSONL file
        f.write(json.dumps(request_obj, ensure_ascii=False) + "\n")

In [10]:
with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/batch_requests_answer_gen_part2.jsonl", "w", encoding="utf-8") as f:
    for idx in range(len(common_indices))[len(common_indices)//2:]:
        common_index = common_indices[idx]
        case1 = case_summary_pairs_list[common_index][0]
        case2 = case_summary_pairs_list[common_index][1]
        citation_reason = o3_mini_response_list[citation_valid_indices_dict[common_index]]
        curr_questions = generated_questions[idx]

        answering_user = f"""Below are the two case descriptions and the set of questions to be answered.

        ---
        Case A:
        {case1}

        Case B:
        {case2}

        Reason for Citation, Possible Similarities and Explanation:
        {citation_reason}

        Questions:
        {curr_questions}
        ---

        Provide your answers now:
        """

        request_obj = {
            "custom_id": f"request-{idx+1}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                    "model": "gpt-4.1-mini",
                    "messages": [
                        {
                            "role": "developer",
                            "content": f"{answering_prompt}"
                        },
                        {
                            "role": "user",
                            "content": f"{answering_user}"
                        }
                    ],
            }
        }
        # Write each object on a new line in the JSONL file
        f.write(json.dumps(request_obj, ensure_ascii=False) + "\n")

In [15]:
with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/batch_requests_answer_gen_part1.jsonl", 'r') as f:
    batch_requests_answer_gen_part1 = [json.loads(line) for line in f]


with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/batch_requests_answer_gen_part2.jsonl", 'r') as g:
    batch_requests_answer_gen_part2 = [json.loads(line) for line in g]


In [63]:
batch_input_file_part1 = client.files.create(
  file=open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/batch_requests_answer_gen_part1.jsonl", "rb"),
  purpose="batch"
)


In [ ]:
# batch_input_file_id_part1 = batch_input_file_part1.id

# client.batches.create(
#     input_file_id=batch_input_file_id_part1,
#     endpoint="/v1/chat/completions",
#     completion_window="24h",
#     metadata={
#       "description": "Generating answers to questions based on case details and citation reasons - Part 1.",
#     }
# )


Batch(id='batch_680deaafc07881909985669b824d064d', completion_window='24h', created_at=1745742511, endpoint='/v1/chat/completions', input_file_id='file-4joKQFnoYeb9uvUDmuFZvP', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1745828911, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Generating answers to questions based on case details and citation reasons - Part 1.'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [4]:
client.batches.retrieve("batch_680deaafc07881909985669b824d064d")

Batch(id='batch_680deaafc07881909985669b824d064d', completion_window='24h', created_at=1745742511, endpoint='/v1/chat/completions', input_file_id='file-4joKQFnoYeb9uvUDmuFZvP', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1745747087, error_file_id=None, errors=None, expired_at=None, expires_at=1745828911, failed_at=None, finalizing_at=1745745662, in_progress_at=1745742523, metadata={'description': 'Generating answers to questions based on case details and citation reasons - Part 1.'}, output_file_id='file-Rvyz8ycBPx4t6KfhVLKaL2', request_counts=BatchRequestCounts(completed=15501, failed=0, total=15501))

In [10]:
file_response = client.files.content('file-Rvyz8ycBPx4t6KfhVLKaL2')
file_response_text = file_response.text

json_strings = file_response_text.strip().splitlines()

# Parse each JSON string into a dictionary and store in a list
response_dicts = [json.loads(line) for line in json_strings]

generated_answers_part1 = [response_dicts[i]['response']['body']['choices'][0]['message']['content'] for i in range(len(response_dicts))]

In [6]:
generated_answers_part1[15500]

'Level 1 – Broad Context:\n1. Yes, both cases utilize ultrasound imaging of the pelvis to evaluate early pregnancy complications, including transvaginal ultrasound and, in Case B, also transabdominal ultrasound.\n2. Yes, both cases focus on assessing the presence of an intrauterine pregnancy alongside an ectopic gestation.\n\nLevel 2 – General Diagnosis Category:\n3. Yes, in both cases, the coexistence of an intrauterine gestation with an adnexal mass contributes to a provisional diagnosis of heterotopic pregnancy.\n4. Yes, both cases presented with clinical signs of hemoperitoneum requiring emergency surgical intervention (laparoscopy and tubal surgery/salpingectomy).\n\nLevel 3 – Specific Features & Findings:\n5. Yes, both cases demonstrated imaging evidence consistent with tubal ectopic pregnancies, including confirmation of ruptured tubes and complex adnexal masses.\n6. Doppler ultrasound was specifically mentioned and used in Case B to assess vascularity and low resistance flow wi

In [28]:
batch_input_file_part2 = client.files.create(
  file=open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/batch_requests_answer_gen_part2.jsonl", "rb"),
  purpose="batch"
)


In [ ]:
# batch_input_file_id_part2 = batch_input_file_part2.id

# client.batches.create(
#     input_file_id=batch_input_file_id_part2,
#     endpoint="/v1/chat/completions",
#     completion_window="24h",
#     metadata={
#       "description": "Generating answers to questions based on case details and citation reasons - Part 2.",
#     }
# )


Batch(id='batch_680e129435308190950894b1eb26d155', completion_window='24h', created_at=1745752724, endpoint='/v1/chat/completions', input_file_id='file-B476YVnoLwuL1mQSdArvPX', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1745839124, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Generating answers to questions based on case details and citation reasons - Part 2.'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [5]:
client.batches.retrieve("batch_680e129435308190950894b1eb26d155")

Batch(id='batch_680e129435308190950894b1eb26d155', completion_window='24h', created_at=1745752724, endpoint='/v1/chat/completions', input_file_id='file-B476YVnoLwuL1mQSdArvPX', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1745772825, error_file_id=None, errors=None, expired_at=None, expires_at=1745839124, failed_at=None, finalizing_at=1745771564, in_progress_at=1745752731, metadata={'description': 'Generating answers to questions based on case details and citation reasons - Part 2.'}, output_file_id='file-Vj78apbM2pF4xG3WZho9zU', request_counts=BatchRequestCounts(completed=15501, failed=0, total=15501))

In [6]:
file_response = client.files.content('file-Vj78apbM2pF4xG3WZho9zU')
file_response_text = file_response.text

json_strings = file_response_text.strip().splitlines()

# Parse each JSON string into a dictionary and store in a list
response_dicts = [json.loads(line) for line in json_strings]

generated_answers_part2 = [response_dicts[i]['response']['body']['choices'][0]['message']['content'] for i in range(len(response_dicts))]

In [7]:
len(generated_answers_part2)

15501

In [19]:
generated_answers = generated_answers_part1 + generated_answers_part2
len(generated_answers)

31002

In [20]:
with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/generated_answers.json', 'w') as f:
    json.dump(generated_answers, f)

In [21]:
with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/generated_answers.json', 'r') as f:
    generated_answers = json.load(f)

In [5]:
matching_uids_xml_paths[common_index]

['/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC4719281/4719281_1/JOCR-3-19.nxml',
 '/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC4719241/4719241_1/JOCR-3-35.nxml']

In [1]:
print(answering_user)

NameError: name 'answering_user' is not defined

In [159]:
response = client.responses.create(
    model="o4-mini",
    reasoning={"effort": "low"},
    input=[
        {
            "role": "developer",
            "content": f"{answering_prompt}"
        },
        {
            "role": "user",
            "content": f"{answering_user}"
        }
    ],
)


In [160]:
print(response.output_text)

Level 1  
1. Yes. Both cases focus on imaging of the shoulder joints.  
2. Yes. Both used plain radiographs (AP view) as the initial imaging.  

Level 2  
3. Yes. Both are traumatic shoulder dislocations.  
4. Yes. Both resulted from falls leading to shoulder dislocations.  

Level 3  
5. Yes. Case A shows asymmetrical dislocations (right posterior, left anterior) versus bilateral anterior dislocations in Case B.  
6. Yes. Case A required the “electric bulb” sign plus axillary and scapular Y views for confirmation, whereas Case B relied on standard AP views.  
7. Yes. Case A had associated greater tuberosity fractures; Case B reports no fractures.  
8. Yes. Both injuries followed falls on an outstretched hand; Case A also involved bystander-induced traction causing the left anterior dislocation.  
9. Yes. Case B had a prior right shoulder dislocation; Case A had no previous dislocations.  
10. Both used closed reduction under general anesthesia (Kocher’s method) and post-reduction immo

In [161]:
print("Input Tokens:", response.usage.input_tokens)
print("Output Tokens:", response.usage.output_tokens)
# print("Reasoning Tokens:", response.usage.output_tokens_details.reasoning_tokens)
print("Total Tokens:", response.usage.total_tokens)

def calc_price(response, input_per_millon, output_per_million):
    input_price = (input_per_millon * response.usage.input_tokens) * 10**-6
    output_price = (output_per_million * response.usage.output_tokens) * 10**-6
    return input_price + output_price


price = calc_price(response, 1.1, 4.4)
print(f"Price: {price:.4f}$")

Input Tokens: 1433
Output Tokens: 460
Total Tokens: 1893
Price: 0.0036$


In [7]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {
            "role": "developer",
            "content": f"{answering_prompt}"
        },
        {
            "role": "user",
            "content": f"{answering_user}"
        }
    ],
)


In [163]:
print(response.output_text)

Level 1: Broad Context  
1. Yes, both cases involve imaging of the shoulder joint as the primary focus.  
2. Yes, both patients were initially evaluated using plain radiographs.  

Level 2: General Diagnosis Category  
3. Yes, both cases can be classified as traumatic shoulder dislocations.  
4. Yes, both cases are associated with a history of falls leading to shoulder dislocations.  

Level 3: Specific Features & Findings  
5. Yes, there is a difference: Case A presents an asymmetrical dislocation pattern (right posterior and left anterior), whereas Case B involves bilateral anterior dislocations.  
6. Yes, imaging in Case A includes findings such as the “electric bulb sign” and uses special views (axillary lateral and scapular Y) to confirm diagnosis, providing greater diagnostic clarity compared to Case B, which primarily reports AP views.  
7. Yes, Case A reports greater tuberosity fractures on both sides alongside dislocations, while Case B does not mention any associated fracture

In [164]:
print("Input Tokens:", response.usage.input_tokens)
print("Output Tokens:", response.usage.output_tokens)
print("Total Tokens:", response.usage.total_tokens)

def calc_price(response, input_per_millon, output_per_million):
    input_price = (input_per_millon * response.usage.input_tokens) * 10**-6
    output_price = (output_per_million * response.usage.output_tokens) * 10**-6
    return input_price + output_price


price = calc_price(response, 0.4, 1.6)
print(f"Price: {price:.4f}$")

Input Tokens: 1434
Output Tokens: 337
Total Tokens: 1771
Price: 0.0011$


In [14]:
!ls /cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC4067084/4067084_1

1472-6823-14-49.nxml  4067084_1_1.txt  4067084_1_2.txt
4067084_1_1.jpg       4067084_1_2.jpg


In [15]:
!cat /cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC4067084/4067084_1/4067084_1_1.txt

Non contrast Computed Tomography (CT) of chest, abdomen and pelvis; transverse and coronal views.

In [16]:
!cat /cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC4067084/4067084_1/4067084_1_2.txt

Hematoxylin and Eosin (H&E) staining of surgically resected tumor. Magnification 40x.

In [18]:
!ls /cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC3457717/3457717_1

3457717_1_1.jpg  3457717_1_2.jpg  CRIM2012-628756.nxml
3457717_1_1.txt  3457717_1_2.txt


In [19]:
!cat /cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC3457717/3457717_1/3457717_1_1.txt

MRI scan demonstrates the recurred tumor mostly involved right posterior fossa.

In [20]:
!cat /cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC3457717/3457717_1/3457717_1_2.txt

Histology and immunohistochemistry of meningeal hemangiopericytoma. (a) H&E 20x A hypercellular tumor with staghorn blood vessels. (b) Immunostaining of CD99 is diffuse and strong positive. (c) Immunostaining of CD 34 reveals variable positive.

**From now on, we'll experiment different encoders to catch similarity between case A answer and case B answer.**

439 (PMC5406076, PMC6689908)  


In [ ]:
from sentence_transformers import SentenceTransformer

In [98]:
# sen1 = "In Case A: No. Although adjuvant therapy was considered, it was deferred by the oncologist due to the very poor prognosis in this young child and possibly due to the aggressive postoperative course; the patient unfortunately succumbed to his illness 5 months postoperatively."
# sen2 = "In Case B: Yes. The adult patient received an aggressive adjuvant treatment regimen including craniospinal irradiation, a local boost, and multiple courses of multiagent chemotherapy, which contributed to a favorable outcome with more than 2 years of recurrence-free follow-up."

sen1 = "In Case A: Yes. The patient underwent MRI with contrast enhancement and additional magnetic resonance spectroscopy (MRS) to assess metabolic features."
sen2 = "In Case B: Yes. The evaluation was primarily with contrast-enhanced MRI, which detailed the solid and cystic components. (While spectroscopy was not performed in Case B, both used MRI with contrast as the key modalit"


sentences = [sen1, sen2]

In [105]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

In [106]:
import torch
import torch.nn.functional as F

idx_a = 0
idx_b = 1

a = torch.tensor(embeddings[idx_a])
b = torch.tensor(embeddings[idx_b])

# F.cosine_similarity expects tensors with an extra dimension if you compute across a batch.
# Here, for a single pair of vectors, we can add a batch dimension.
a_2d = a.unsqueeze(0)  # shape (1, 3)
b_2d = b.unsqueeze(0)  # shape (1, 3)

# dim=1 means we treat each row as a vector and compute similarity along features (columns).
similarity = F.cosine_similarity(a_2d, b_2d, dim=1)

print(f"Cosine similarity between {idx_a} and {idx_b}:", similarity.item())


Cosine similarity between 0 and 1: 0.6678763031959534


In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

[[-1.14403320e-02  3.55402678e-02 -6.49787188e-02  2.85187662e-02
  -6.95913956e-02 -2.47340854e-02  6.67356849e-02  3.89381163e-02
  -4.09621000e-02  5.15950322e-02  9.18183848e-02  1.84400886e-01
  -8.50416906e-03  4.07558121e-02 -1.89403556e-02  7.87346438e-02
   9.67198089e-02  4.24947850e-02  2.16073897e-02  2.70406082e-02
  -6.16211854e-02  2.72715110e-02 -6.41249167e-03  5.95295690e-02
  -9.74445604e-03 -6.67739883e-02 -2.21402757e-02  2.50036027e-02
  -2.53274776e-02  3.11616994e-02 -2.58509186e-03 -6.13423884e-02
  -6.03324510e-02 -5.02152406e-02 -3.56790051e-02 -5.37888287e-03
  -1.13530075e-02  8.39234889e-02 -3.21301930e-02  7.15836436e-02
   2.51444299e-02  7.36918971e-02 -8.20456222e-02  1.09235831e-02
   5.19106314e-02 -1.35764644e-01 -6.09100908e-02 -7.37459958e-02
  -6.29536668e-03  2.71262359e-02 -4.73186485e-02 -3.21800746e-02
   1.11923525e-02  1.95217784e-02 -6.79730922e-02 -4.96086739e-02
  -5.62511086e-02  2.56017894e-02 -7.01818708e-03  4.91014607e-02
  -5.04838

In [77]:
import torch
import torch.nn.functional as F

idx_a = 0
idx_b = 1

a = torch.tensor(embeddings[idx_a])
b = torch.tensor(embeddings[idx_b])

# F.cosine_similarity expects tensors with an extra dimension if you compute across a batch.
# Here, for a single pair of vectors, we can add a batch dimension.
a_2d = a.unsqueeze(0)  # shape (1, 3)
b_2d = b.unsqueeze(0)  # shape (1, 3)

# dim=1 means we treat each row as a vector and compute similarity along features (columns).
similarity = F.cosine_similarity(a_2d, b_2d, dim=1)

print(f"Cosine similarity between {idx_a} and {idx_b}:", similarity.item())

Cosine similarity between 0 and 1: 0.6878408789634705


In [84]:
model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
embeddings = model.encode(sentences)

In [85]:
import torch
import torch.nn.functional as F

idx_a = 0
idx_b = 1

a = torch.tensor(embeddings[idx_a])
b = torch.tensor(embeddings[idx_b])

# F.cosine_similarity expects tensors with an extra dimension if you compute across a batch.
# Here, for a single pair of vectors, we can add a batch dimension.
a_2d = a.unsqueeze(0)  # shape (1, 3)
b_2d = b.unsqueeze(0)  # shape (1, 3)

# dim=1 means we treat each row as a vector and compute similarity along features (columns).
similarity = F.cosine_similarity(a_2d, b_2d, dim=1)

print(f"Cosine similarity between {idx_a} and {idx_b}:", similarity.item())

Cosine similarity between 0 and 1: 0.9592615365982056


In [101]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/nli-roberta-base')
# scores = model.predict([('A man is eating pizza', 'A man eats something'), ('A black race car starts up in front of a crowd of people.', 'A man is driving down a lonely road.')])
scores = model.predict([(sen1, sen2)])

#Convert scores to labels
label_mapping = ['contradiction', 'entailment', 'neutral']
labels = [label_mapping[score_max] for score_max in scores.argmax(axis=1)]


In [102]:
labels

['contradiction']

In [ ]:
import cv2
import numpy as np

# Load the compound figure
image = cv2.imread('compound_figure.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding: adjust the threshold value depending on your image's background
_, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

# Use morphological closing to fill gaps in the sub-images' boundaries
kernel = np.ones((5, 5), np.uint8)
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# Find external contours in the processed image
contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Loop over contours, filter out small regions (noise), and extract sub-images
sub_images = []
for i, cnt in enumerate(contours):
    x, y, w, h = cv2.boundingRect(cnt)
    # Filter out very small contours that might be noise
    if w > 50 and h > 50:
        sub_img = image[y:y+h, x:x+w]
        sub_images.append(sub_img)
        cv2.imwrite(f'sub_image_{i}.jpg', sub_img)

print(f"Extracted {len(sub_images)} sub-images.")
